In [1]:
# -- HIDDEN PLINKO v1.1 --
#  Imports and global setup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import random
import io

from IPython.display import display, HTML, clear_output, Javascript

import ipywidgets as widgets

# Ensure export directory
if not os.path.exists("exports"):
    os.makedirs("exports")

# Globals
puck_positions = []
final_positions = []
full_logs = []
export_ready = False
run_counter = 0
loaded_datasets = []
notes_text = ""



In [2]:
# -- UI WIDGETS & CONTROLS --

# Core Controls
num_pucks_slider = widgets.IntSlider(value=500, min=0, max=50000, step=100, description='Pucks')
def check_large_run(change):
    if change['new'] > 10000:
        print("⚠️ Warning: Running more than 10,000 pucks may slow down performance.")
    elif change['new'] < 1:
        print("⚠️ Warning: Must run at least 1 puck.")

num_pucks_slider.observe(check_large_run, names='value')

spread_width_slider = widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description='Spread Width')
auto_scale_checkbox = widgets.Checkbox(value=True, description='Auto-Scale Board')

# Bias + Symmetry
bias_toggle = widgets.Checkbox(value=True, description='Bias ON')
bias_strength_slider = widgets.FloatSlider(value=5, min=0, max=20, step=0.5, description='Bias Strength (°)')
lobe_slider = widgets.IntSlider(value=3, min=2, max=12, description='Number of Lobes')
symmetry_type_dropdown = widgets.Dropdown(
    options=['Rotational', 'Elliptical', 'Spoked', 'Randomized', 'Mirrored'],
    value='Rotational', description='Symmetry Type'
)
symmetry_strength_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, description='Symmetry Strength')
dynamic_lobes_checkbox = widgets.Checkbox(value=False, description='Dynamic Lobes')

# Field
field_strength_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='Field X')
dynamic_field_checkbox = widgets.Checkbox(value=False, description='Dynamic Field')
dynamic_field_strength_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.005, description='Dynamic Field Strength')
dynamic_symmetry_strength_slider = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, description='Dyn Sym Strength')

# Notes
notes_textbox = widgets.Textarea(
    value='', placeholder='Enter experiment notes here...',
    description='Notes:', layout=widgets.Layout(width='80%', height='100px')
)

# Logging
logging_mode_dropdown = widgets.Dropdown(
    options=['Off', 'On', 'All', 'Custom'], description='Logging Mode'
)

# Buttons
run_button = widgets.Button(description='Run Simulation', button_style='success')
save_button = widgets.Button(description='Save Export', button_style='info')
## clear_button = widgets.Button(description='Clear Results', button_style='danger')
plot_button = widgets.Button(description='Plot Results', button_style='info')
clear_plot_button = widgets.Button(description='Clear Plot', button_style='danger')
bins_slider = widgets.IntSlider(value=30, min=5, max=100, step=1, description='Bins')
##compare_current_file_button = widgets.Button(description='Compare Current vs File', button_style='primary')
##compare_two_files_button = widgets.Button(description='Compare Two Files', button_style='primary')
##clear_comparison_button = widgets.Button(description='Clear Comparison', button_style='danger')
logging_status_output = widgets.Output()
plot_output = widgets.Output()

#Widgets
upload_widget_one = widgets.FileUpload(accept='.csv', multiple=False)
upload_widget_two = widgets.FileUpload(accept='.csv', multiple=True)
settings_upload = widgets.FileUpload(accept='.csv', multiple=False)
upload_settings_button = widgets.Button(description="Upload Settings CSV", button_style="info")
settings_upload_output = widgets.Output()




In [3]:
# -- SETTINGS UPLOAD TOOL --



def show_settings_upload(_=None):
    with settings_upload_output:
        clear_output()
        print("📥 Upload a CSV to apply experiment settings:")
        display(settings_upload)
    display(Javascript("window.scrollTo(0, document.body.scrollHeight);"))

def apply_settings_from_csv(file_info):
    try:
        content = io.BytesIO(file_info['content'])
        df = pd.read_csv(content)
        if df.empty:
            print("⚠️ No rows found in uploaded CSV.")
            return

        row = df.iloc[0]  # Only first row for now

        # Assign values to widgets
        num_pucks_slider.value = int(row['Num_Pucks'])
        spread_width_slider.value = float(row['Spread_Width'])
        bias_toggle.value = bool(row['Bias_ON'])
        bias_strength_slider.value = float(row['Bias_Strength'])
        lobe_slider.value = int(row['Num_Lobes'])
        symmetry_type_dropdown.value = row['Symmetry_Type']
        symmetry_strength_slider.value = float(row['Symmetry_Strength'])
        dynamic_lobes_checkbox.value = bool(row['Dynamic_Lobes'])
        field_strength_slider.value = float(row['Field_Strength'])
        dynamic_field_checkbox.value = bool(row['Dynamic_Field'])
        dynamic_field_strength_slider.value = float(row['Dynamic_Field_Strength'])
        dynamic_symmetry_strength_slider.value = float(row['Dynamic_Symmetry_Strength'])
        notes_textbox.value = str(row.get('Experiment_Notes', ''))

        print("✅ Settings applied from CSV.")
        refresh_plot_status()

    except Exception as e:
        print(f"❌ Failed to apply settings: {e}")

def handle_settings_upload(change):
    print("📥 File detected. Handling upload...")

    files = settings_upload.value

    # Show what the raw data structure looks like
    print("🧪 Upload widget value contents:", files)

    if isinstance(files, dict):
        for name, file_info in files.items():
            print(f"📄 Applying: {name}")
            apply_settings_from_csv(file_info)

    elif isinstance(files, tuple):
        for i, file_info in enumerate(files):
            print(f"📄 Applying uploaded file {i + 1}")
            apply_settings_from_csv(file_info)

    else:
        print("⚠️ Unexpected upload structure:", type(files))


settings_upload.observe(handle_settings_upload, names='value')

# -- BATCH RUNNER --

batch_upload = widgets.FileUpload(accept='.csv', multiple=False)
batch_output = widgets.Output()

def apply_settings_row(row):
    num_pucks_slider.value = int(row['Num_Pucks'])
    spread_width_slider.value = float(row['Spread_Width'])
    bias_toggle.value = bool(row['Bias_ON'])
    bias_strength_slider.value = float(row['Bias_Strength'])
    lobe_slider.value = int(row['Num_Lobes'])
    symmetry_type_dropdown.value = row['Symmetry_Type']
    symmetry_strength_slider.value = float(row['Symmetry_Strength'])
    dynamic_lobes_checkbox.value = bool(row['Dynamic_Lobes'])
    field_strength_slider.value = float(row['Field_Strength'])
    dynamic_field_checkbox.value = bool(row['Dynamic_Field'])
    dynamic_field_strength_slider.value = float(row['Dynamic_Field_Strength'])
    dynamic_symmetry_strength_slider.value = float(row['Dynamic_Symmetry_Strength'])
    notes_textbox.value = str(row.get('Experiment_Notes', ''))

def run_batch_experiments(change):
    with batch_output:
        clear_output()
        files = batch_upload.value

        if isinstance(files, dict):
            files = list(files.values())

        if not isinstance(files, (tuple, list)):
            print("⚠️ Unexpected upload format:", type(files))
            return

        for file_info in files:
            df = pd.read_csv(io.BytesIO(file_info['content']))
            print("📄 Loaded batch file (tuple or dict format)")
            for i, row in df.iterrows():
                print(f"🔁 Running experiment {i + 1}/{len(df)}...")
                apply_settings_row(row)

                # Clear global logs
                global final_positions, full_logs, export_ready
                final_positions = []
                full_logs = []

                # Run simulation
                for puck_id in range(int(row['Num_Pucks'])):
                    simulate_single_puck(puck_id)

                print(f"🧪 Final positions collected: {len(final_positions)}")
                if final_positions:
                    export_ready = True
                    save_results(None)
                else:
                    print("⚠️ No data collected — skipping export.")
            print("✅ All batch experiments complete.")

batch_upload.observe(run_batch_experiments, names='value')

batch_button = widgets.Button(description="Run Batch CSV", button_style="warning")
def show_batch_upload(_=None):
    with batch_output:
        clear_output()
        print("📥 Upload a CSV with multiple experiments to run in sequence:")
        display(batch_upload)
batch_button.on_click(show_batch_upload)




In [4]:
# -- LIVE LOGGING STATUS --
def update_logging_status(_=None):
    with logging_status_output:
        clear_output()
        if logging_mode_dropdown.value == 'Off':
            display(HTML('<span style="color:red; font-weight:bold;">❌ Logging is off. No data will be saved.</span>'))
        else:
            display(HTML('<span style="color:green; font-weight:bold;">✅ Logging enabled: logs will be saved with experiment.</span>'))

# -- AUTO-NOTATION FOR EXPORT --
def collect_auto_notation():
    settings = {
        'Num_Pucks': num_pucks_slider.value,
        'Spread_Width': spread_width_slider.value,
        'Bias_ON': bias_toggle.value,
        'Bias_Strength': bias_strength_slider.value,
        'Num_Lobes': lobe_slider.value,
        'Symmetry_Type': symmetry_type_dropdown.value,
        'Symmetry_Strength': symmetry_strength_slider.value,
        'Dynamic_Lobes': dynamic_lobes_checkbox.value,
        'Field_Force_Profile': 'Dynamic' if dynamic_field_checkbox.value else 'Constant',
        'Field_Strength': field_strength_slider.value,
        'Dynamic_Field': dynamic_field_checkbox.value,
        'Dynamic_Field_Strength': dynamic_field_strength_slider.value,
        'Dynamic_Symmetry_Strength': dynamic_symmetry_strength_slider.value,
        'Timestamp': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'Experiment_Notes': notes_textbox.value
    }
    return settings


In [5]:
# -- SIMULATION ENGINE --

def handle_clear_click(_):
    clear_output(wait=True)
    display_controls()
    refresh_plot_status()

def rotate_vector(vec, angle_deg):
    angle_rad = np.deg2rad(angle_deg)
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad),  np.cos(angle_rad)]
    ])
    return rotation_matrix @ vec

def field_force(time_step):
    """ Returns external field force vector. Handles dynamic field if enabled. """
    if dynamic_field_checkbox.value:
        dynamic = field_strength_slider.value * np.sin(time_step * dynamic_field_strength_slider.value)
        return np.array([dynamic, 0.0])
    else:
        return np.array([field_strength_slider.value, 0.0])

def compute_internal_bias(puck_spin_angle, impact_angle, time_step=0):
    lobes = lobe_slider.value
    symmetry_strength = symmetry_strength_slider.value
    
    if dynamic_lobes_checkbox.value:
        symmetry_strength += dynamic_symmetry_strength_slider.value * np.sin(time_step * 0.1)
        lobes += int(np.sin(time_step * 0.05) * 2)
        lobes = max(2, lobes)

    lobe_sector = 360 / lobes
    relative_angle = (impact_angle - puck_spin_angle) % 360
    distance_to_nearest_lobe = min(relative_angle % lobe_sector, lobe_sector - (relative_angle % lobe_sector))
    normalized_distance = distance_to_nearest_lobe / (lobe_sector / 2)

    bias = bias_strength_slider.value * (1 - normalized_distance)

    if symmetry_type_dropdown.value == 'Elliptical':
        bias *= 1.5 if np.abs(np.sin(np.deg2rad(impact_angle))) > 0.7 else 0.5
    elif symmetry_type_dropdown.value == 'Spoked':
        bias *= random.uniform(0.5, 1.5)
    elif symmetry_type_dropdown.value == 'Randomized':
        bias *= random.uniform(0.2, 2.0)
    elif symmetry_type_dropdown.value == 'Mirror':
        if impact_angle > 180:
            bias *= -1

    return np.random.choice([-1, 1]) * bias

def simulate_single_puck(puck_id=0):
    spread = spread_width_slider.value
    position = np.array([np.random.uniform(-spread/2, spread/2), 0.0])
    velocity = np.array([0.0, -1.0])
    puck_spin_angle = np.random.uniform(0, 360)
    
    num_rows = 10
    peg_spacing = 1.0
    full_trajectory = []

    for row in range(num_rows):
        offset = (row % 2) * (peg_spacing / 2)
        peg_x = round((position[0] - offset) / peg_spacing) * peg_spacing + offset
        peg_y = -(row * peg_spacing)

        peg_position = np.array([peg_x, peg_y])
        impact_vector = position - peg_position
        impact_normal = impact_vector / np.linalg.norm(impact_vector)

        velocity = velocity - 2 * np.dot(velocity, impact_normal) * impact_normal

        if bias_toggle.value:
            impact_angle = np.rad2deg(np.arctan2(impact_normal[1], impact_normal[0])) % 360
            bias_angle = compute_internal_bias(puck_spin_angle, impact_angle, row)
            velocity = rotate_vector(velocity, bias_angle)

        velocity += field_force(row) * 0.1
        position += velocity * peg_spacing * 0.5

        if logging_mode_dropdown.value in ['All', 'Custom']:
            full_trajectory.append({
                'puck_id': puck_id,
                'row': row,
                'pos_x': position[0],
                'pos_y': position[1],
                'vel_x': velocity[0],
                'vel_y': velocity[1]
            })

    final_positions.append(position[0])
    if full_trajectory:
        full_logs.append(full_trajectory)

def run_simulation(_=None):
    global final_positions, full_logs, run_counter, export_ready
    final_positions = []
    full_logs = []

    progress = widgets.IntProgress(value=0, min=0, max=num_pucks_slider.value, description='0% done', bar_style='info')
    display(progress)

    for puck_id in range(num_pucks_slider.value):
        simulate_single_puck(puck_id)
        if puck_id % 100 == 0:
            progress.value = puck_id
            percent_done = int((puck_id / num_pucks_slider.value) * 100)
            progress.description = f"{percent_done}% done"

    progress.value = num_pucks_slider.value
    progress.bar_style = 'success'
    progress.description = "✅ Complete"
    export_ready = True
    run_counter += 1
    update_logging_status()
    plot_results()


In [6]:
# -- PLOT + CLEAR FUNCTIONS --
def refresh_plot_status():
    with plot_output:
        clear_output()
        if not final_positions:
            print("⚠️ No data to plot. Run the simulation first.")
        else:
            print(f"✅ {len(final_positions)} pucks simulated. Click 'Plot Results' to view.")


def plot_results(_=None):
    with plot_output:
        clear_output(wait=True)
                
        plt.figure(figsize=(10, 5))
        plt.hist(final_positions, bins=bins_slider.value, alpha=0.7, edgecolor='black')
        plt.title("Final X Positions of Pucks")
        plt.xlabel("X Position")
        plt.ylabel("Frequency")
        plt.grid(True)
        plt.show()

def clear_plot(_=None):
    clear_output(wait=True)
    refresh_plot_status()
   
    print("🧹 Cleared plot.")



# -- EXPORT FUNCTION --

def save_results(_=None):
    global run_counter
    if not export_ready:
        print("No simulation run yet!")
        return
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    export_filename = f"exports/HPI_export_{timestamp}.csv"
    
    df = pd.DataFrame({'Final_X_Position': final_positions})
    settings = collect_auto_notation()
    for key, value in settings.items():
        df[key] = value
    df.to_csv(export_filename, index=False)
    print(f"✅ Exported results to {export_filename}")


In [7]:
# -- COMPARISON TOOLS 
# Upload widgets defined globally to persist and work correctly
upload_widget_one = widgets.FileUpload(accept='.csv', multiple=False)
upload_widget_two = widgets.FileUpload(accept='.csv', multiple=True)

# --- Handler: Compare Current Run vs Uploaded File ---
def handle_single_file(change):
    if not final_positions:
        print("⚠️ No current simulation data to compare.")
        return

    for name, file_info in upload_widget_one.value.items():
        content = io.BytesIO(file_info['content'])
        df = pd.read_csv(content)
        loaded_positions = df['Final_X_Position'].values

        plt.figure(figsize=(10, 5))
        plt.hist(final_positions, bins=30, alpha=0.5, label='Current Run')
        plt.hist(loaded_positions, bins=30, alpha=0.5, label='Loaded File')
        plt.title('Comparison: Current vs Loaded')
        plt.xlabel('X Position')
        plt.ylabel('Number of Pucks')
        plt.legend()
        plt.grid(True)
        plt.show()

upload_widget_one.observe(handle_single_file, names='value')

# --- Handler: Compare Two Uploaded Files ---
def handle_two_files(change):
    files = list(upload_widget_two.value.values())
    if len(files) < 2:
        print("⚠️ Please upload two CSV files.")
        return

    df1 = pd.read_csv(io.BytesIO(files[0]['content']))
    df2 = pd.read_csv(io.BytesIO(files[1]['content']))
    pos1 = df1['Final_X_Position'].values
    pos2 = df2['Final_X_Position'].values

    plt.figure(figsize=(10, 5))
    plt.hist(pos1, bins=30, alpha=0.5, label='File 1')
    plt.hist(pos2, bins=30, alpha=0.5, label='File 2')
    plt.title('Comparison: File 1 vs File 2')
    plt.xlabel('X Position')
    plt.ylabel('Number of Pucks')
    plt.legend()
    plt.grid(True)
    plt.show()

upload_widget_two.observe(handle_two_files, names='value')

# --- UI Buttons trigger uploads ---
def compare_current_vs_file(_=None):
    print("📂 Upload a CSV file to compare with the current simulation.")
    display(upload_widget_one)

def compare_two_files(_=None):
    print("📂 Upload two CSV files to compare.")
    display(upload_widget_two)

def clear_comparisons(_=None):
    clear_output(wait=True)
    refresh_plot_status()
    print("✅ Cleared loaded comparisons.")



In [8]:
# -- DISPLAY CONTROLS --

def display_controls():
        display(widgets.VBox([
        widgets.HBox([run_button, save_button]),
        widgets.HBox([plot_button, clear_plot_button, bins_slider]),
        ##widgets.HBox([compare_current_file_button, compare_two_files_button, clear_comparison_button]),
        logging_mode_dropdown,
        logging_status_output,
        notes_textbox,
        plot_output,
        settings_upload_output,
        batch_button,
        batch_output,
    ]))


# -- FULL UI LAUNCHER --

def launch_playground():
    display(widgets.HTML('<h2><b> Hidden Plinko Experimental Playground</b></h2>'))
    display(widgets.HTML('<i>Explore deterministic chaos through structured substrate simulation.</i>'))
    display(widgets.VBox([
        widgets.HTML('<h4>Setup Your Experiment:</h4>'),
        widgets.HBox([num_pucks_slider, spread_width_slider, auto_scale_checkbox]),
        widgets.HBox([bias_toggle, bias_strength_slider, lobe_slider]),
        widgets.HBox([symmetry_type_dropdown, symmetry_strength_slider, dynamic_lobes_checkbox]),
        widgets.HBox([field_strength_slider, dynamic_field_checkbox]),
        widgets.HBox([dynamic_symmetry_strength_slider, dynamic_field_strength_slider]),
        widgets.HTML('<h4>Controls & Output:</h4>'),
    ]))
    refresh_plot_status()
    update_logging_status()
    display_controls()
    display(widgets.HBox([upload_settings_button]))

        

In [9]:
# -- BUTTON LINKING --
run_button.on_click(run_simulation)
plot_button.on_click(plot_results)
##clear_button.on_click(handle_clear_click)
clear_plot_button.on_click(clear_plot)
save_button.on_click(save_results)
##compare_current_file_button.on_click(compare_current_vs_file)
##compare_two_files_button.on_click(compare_two_files)
##clear_comparison_button.on_click(clear_comparisons)
logging_mode_dropdown.observe(update_logging_status, names='value')
plot_output = widgets.Output()
upload_settings_button.on_click(show_settings_upload)
upload_settings_button = widgets.Button(description="Upload Settings CSV", button_style="info")
upload_settings_button.on_click(show_settings_upload)


# -- AUTO-LAUNCH --
launch_playground()


HTML(value='<h2><b> Hidden Plinko Experimental Playground</b></h2>')

HTML(value='<i>Explore deterministic chaos through structured substrate simulation.</i>')

## Hidden Plinko Roadmap

### v1.1 (Current Version)
- CSV import of batch settings
- Dynamic substrate & field bias
- Realtime plotting
- Logging & metadata export
- Fully interactive playground

### Planned v2 Features
- Comparison tools
- Parameter sweep automation
- Preset configuration dropdowns
- Fractal bias or terrain-style substrate
- Magnetic field gradients

### Future v3 Ideas
- Animation of puck paths
- Cosmology-style curvature substrates
- Symbolic Lagrangian expression of structure
- Networked simulation grid (multi-agent substrates)

---

Built with curiosity by Gardener Dave 🌱
